In [1]:
#!/usr/bin/env python
# coding: utf-8
get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold

import gc

from tqdm import tqdm_notebook

import warnings
warnings.filterwarnings(action='ignore')


import torch
print(torch.__version__)
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from torchsummary import summary

print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device('cpu')
device

/home/aiden/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


2.3.2
1.3.1
GeForce RTX 2070 SUPER
True


device(type='cuda', index=0)

In [2]:
print(os.getcwd())

base_path = '.'

df_train = pd.read_csv(os.path.join(base_path , 'input/train.csv'), index_col=0)
df_test = pd.read_csv(os.path.join(base_path , 'input/public_test.csv'), index_col=0)
df_test['smishing'] = -1

df_fea = pd.concat([df_train, df_test])
df_fea.shape

/home/aiden/src/dacon_14


(297571, 3)

In [3]:
dfs = []

### Mecab

In [4]:
# mecab = Mecab()
# # df_space['morphs'] = df_space['spacing'].apply(lambda x: mecab.morphs(x))
# df_fea['nouns'] = df_fea['text'].apply(lambda x: mecab.nouns(x))

# df_fea['nouns_str'] = df_fea['nouns'].apply(lambda x: ' '.join(x))

In [5]:
# df_fea.to_pickle('df_fea.pkl')

In [6]:
df_fea = pd.read_pickle('data/df_fea.pkl')#[:1000]

In [17]:
def _toidx(x):
    return [int(vocab_dict[x[i]]) if i < len(x) else 0 for i in range(max_len)]

def toidx(src_col, max_len):
    idx_col_nm = '{}_{}_idx'.format(src_col, max_len)

    vocab_set = set()
    _ = df_fea[src_col].apply(lambda x: [vocab_set.add(c) for c in x])

    vocab_dict = {v: i+1 for i, v in enumerate(vocab_set)}
    vocab_dim = len(vocab_dict.keys()) + 1

    def toidx(x):
        return [int(vocab_dict[x[i]]) if i < len(x) else 0 for i in range(max_len)]

    print(vocab_dim, max_len)
    
    return idx_col_nm, df_fea[src_col].apply(toidx), vocab_dim, max_len

In [18]:
src_col = 'nouns'
max_len = 100

c, d, vocab_dim, max_len = toidx(src_col, max_len)
df_fea[c] = d
c

24161 100


'nouns_100_idx'

In [ ]:
# src_col = 'text'
# max_len = 100

In [19]:
df_merged = df_fea[[c, 'smishing']]
df_merged

,nouns_100_idx,smishing
id,,
0,"[17437, 14804, 23106, 3204, 14669, 0, 0, 0, 0,...",0
1,"[8288, 8847, 21933, 16764, 21199, 9233, 0, 0, ...",0
2,"[10875, 24138, 16764, 21103, 12421, 7645, 4683...",0
4,"[24138, 10875, 16764, 17896, 2824, 9280, 16371...",0
5,"[11480, 4155, 3204, 2824, 0, 0, 0, 0, 0, 0, 0,...",0
...,...,...
341621,"[10499, 24138, 10875, 24138, 6868, 23864, 4561...",-1
341622,"[24138, 16371, 17896, 4440, 14397, 22911, 1174...",-1
341623,"[24138, 16371, 16764, 17896, 24009, 14397, 102...",-1


In [20]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 297571 entries, 0 to 341625
Data columns (total 2 columns):
nouns_100_idx    297571 non-null object
smishing         297571 non-null int64
dtypes: int64(1), object(1)
memory usage: 6.8+ MB


In [21]:
merged_ts = '{}_{}_{}'.format(datetime.now().strftime('%Y%m%dT%H%M%S'), str(max_len), str(vocab_dim))
print(merged_ts)
joblib.dump(df_merged.loc[df_merged['smishing'] != -1,:], 'data/df_merged_{}_train.pkl'.format(merged_ts))

joblib.dump(df_merged.loc[df_merged['smishing'] == 1,:], 'data/df_merged_{}_train_pos.pkl'.format(merged_ts))

joblib.dump(df_merged.loc[df_merged['smishing'] == -1,:], 'data/df_merged_{}_test.pkl'.format(merged_ts))

20200101T002838_100_24161


['data/df_merged_20200101T002838_100_24161_test.pkl']

In [ ]:
gc.collect()

exit(0)